Copyright 2024 Lehi Gracia

### Save?

In [1]:
save_to_mongo = False

import subprocess, os
if save_to_mongo:
    mongod = subprocess.Popen(
        ['mongod', '--dbpath', os.path.expanduser(os.getenv('MONGODB_PATH')), '--logpath', os.path.expanduser(os.getenv('MONGODB_LOG'))]
    )

### MARKETS

In [2]:

import warnings
warnings.filterwarnings('ignore')

from yfinance_utils import list_utils, file_utils, constants
import yfinance 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA



COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

data = yfinance.download(symbols, period='1y',auto_adjust=False )
gains = {}
for symbol in symbols:
    df1 = pd.DataFrame()
    df1 = data.loc[:,(slice(None),symbol)]
    df1.columns = COLUMNS
    df1['sma'] = TA.SMA(df1, period=50)
    df1 = df1[50:]
    df1.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=df1['sma'], x=df1['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:6}: {names[symbol]}')



[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:
SPY :  3.23: S&P 500
DIA :  2.77: DOW JONES
QQQ :  2.43: NASDAQ 100


### Implied Volaility

In [3]:
import yfinance
from finta import TA
from plotly.subplots import make_subplots
_t = yfinance.Ticker('^VIX')

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
_d = _t.history(period='1y')
_d['sma'] = TA.SMA(_d, period=30)

_f=make_subplots(specs=[[{"secondary_y": True}]])
_f.update_layout(title=f"Market Volatility (VIX)", title_x=0.5, width=600, height=400)
_d.reset_index(inplace=True)
_f.add_trace(
    go.Scatter(y=_d['Close'], x=_d['Date'], name="VIX"),
    secondary_y=False
)
_f.add_trace(
    go.Scatter(y=_d['sma'], x=_d['Date'], name=f'sma'),
    secondary_y=False
)
_f.show()

### Sectors

In [4]:

from yfinance_utils import list_utils
from datetime import datetime

symbols = list_utils.get_sectors()
names = list_utils.sectors
COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
NUMBER_OF_TOP_PERFORMERS_IN_MARKETS = 3

fig2=make_subplots(rows=3, cols=1)
fig2.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_MARKETS} SECTORS", title_x=0.5, width=800, height=600)

top = list_utils.get_top_performers(symbols, size=NUMBER_OF_TOP_PERFORMERS_IN_MARKETS)
_list = [i[0] for i in top]

data = yfinance.download(_list, period='1y', auto_adjust=False)
counter=1
for symbol in _list:
    _d2 = pd.DataFrame()
    _d2 = data.loc[:,(slice(None),symbol)]
    _d2.columns = COLUMNS
    _d2['sma'] = TA.SMA(_d2, period=50)
    _d2.reset_index(inplace=True)
    fig2.add_trace(
        go.Scatter(y=_d2['Close'], x=_d2['Date'], name=names[symbol]),
        secondary_y=False,
        row=counter, col=1
    )
    fig2.add_trace(
        go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
        secondary_y=False,
        row=counter, col=1
    )
    counter = counter + 1
fig2.show()

today = datetime.today().strftime('%Y-%m-%d')
print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol, value in top:
    if save_to_mongo:
        file_utils.save_to_mongo({"DATE":today, "TICK":symbol, "PERFORMANCE":value, "SECTOR":names[symbol]}, "top_sectors")



[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:


###  Getting 5 best performing stocks in each of top 3 sectors

In [5]:
# Best performance stock in sectors

from yfinance_utils import list_utils
import numpy as np

NUMBER_OF_TOP_PERFORMERS_IN_SECTORS = 5
SMA_PERIOD = 50
_sym = {}
for symbol, value in top:
    _l = []
    for s in list_utils.SECTORS[symbol].keys():
        _l.append(s)
    _tp = list_utils.get_top_performers(_l, days=30, size=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS)
    _l = [i[0] for i in _tp]
    _sym[symbol] = _l
    
for i in list(_sym.keys()):
    
    _f=make_subplots(rows=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS, cols=1)
    _f.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_SECTORS} from {names[i]}", title_x=0.5, width=600, height=1000)
    
    data = yfinance.download(_sym[i], period='1y', auto_adjust=False)
    counter = 1
    for symbol in _sym[i]:
        _d2 = pd.DataFrame()
        _d2 = data.loc[:,(slice(None),symbol)]
        _d2.columns = COLUMNS
        _d2['sma'] = TA.SMA(_d2, period=SMA_PERIOD)
        _d2['pct'] = _d2['sma'].pct_change()
        _d2['pct_close'] = _d2['Close'] * _d2['pct'] * 25

        _d2["Color"] = np.where(_d2["pct_close"] < 0, 'red', 'green')

        _d2.reset_index(inplace=True)
        _f.add_trace(
            go.Scatter(y=_d2['Close'], x=_d2['Date'], name=symbol,showlegend=False),
            col=1, row=counter
        )
        _f.add_trace(
            go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma {SMA_PERIOD}', showlegend=False),
            col=1, row=counter
        )
        _f.add_trace(
            go.Bar(y=_d2['pct_close'], x=_d2['Date'], name=f'change', marker_color=_d2['Color'], showlegend=False),
            col=1, row=counter
        )
        _f.update_yaxes(side='right')
        price = f"{symbol} - Date: {today}, Open:{round(_d2['Open'].iloc[-1],2)}, High:{round(_d2['High'].iloc[-1],2)}, Low:{round(_d2['Low'].iloc[-1],2)}, Close: {round(_d2['Close'].iloc[-1],2)}"
        _f.add_annotation(xref='x domain',
                        yref='y domain',
                        x=0.01,
                        y=1,
                        text=price, 
                        showarrow=False,
                        font=dict(size=10),
                        row=counter, col=1)
        counter = counter + 1
    _f.show()

if save_to_mongo:
    _sym['DATE'] = datetime.today().strftime('%Y-%m-%d')
    file_utils.save_to_mongo(_sym, "top_performers")

[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


### Top Analysts Ratings

In [6]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)

client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':yesterday, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2).sort_values(by="TOTAL", axis=0, ascending=False).drop_duplicates(subset=["TICK"], keep='last').iloc[:20]
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)


ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 68b0091db9468c5c077408e1, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

### Gap up close high

In [ ]:
# Gap up close high

res = collection.find(
                 {'DATE':yesterday, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)

             TICK    HIGH   CLOSE    OPEN  CLOSE -1
DATE                                               
2025-01-21   ANET  121.92  121.56  121.50    119.90
2025-01-21     MO   52.18   52.02   51.47     51.08
2025-01-21    HAL   30.26   30.20   29.53     29.39
2025-01-21    XPO  140.00  139.39  138.31    138.09
2025-01-21  BRK-B  473.17  471.00  468.57    464.71
2025-01-21   TSLA  432.98  432.81  424.07    421.50
2025-01-21    XLY  231.50  231.29  230.90    229.03
2025-01-21    XLE   94.00   93.72   93.48     93.24
2025-01-21    XOM  112.18  112.05  111.47    111.03
2025-01-21    CMS   68.94   68.65   68.30     67.84
2025-01-21    MDT   88.70   88.45   88.38     87.25
2025-01-21    FCX   40.94   40.76   40.30     40.09
2025-01-21     FI  210.78  210.00  208.98    207.80
2025-01-21      B   47.45   47.45   47.39     47.37
2025-01-21      K   81.82   81.79   81.69     81.49
2025-01-21    PDD  108.40  108.07  104.15    101.50
2025-01-21   MRVL  126.11  125.86  123.78    119.92
2025-01-21  

### Price drops and jumps

In [ ]:
# Price Ups and Downs

res = collection.find(
                 {'DATE':yesterday, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)

            TICK   CLOSE  % PRICE JUMP  % VOLUME -1  % VOLUME -2
DATE                                                            
2025-01-21  MPWR  637.09          5.04        13.57       115.42
2025-01-21  XPEV   14.92          6.72        69.29       -28.38
2025-01-21   SMR   23.86         13.62        30.64        76.93
2025-01-21  MRNA   35.80          5.79        57.36         8.30
2025-01-21   VST  179.30          6.08        24.77        43.01
2025-01-21   VVX   54.79          5.06       177.45       -44.10
2025-01-21   NNE   26.63         11.10       158.76         6.99
2025-01-21  QRVO   87.15         11.73       -51.19       505.75
2025-01-21  OKLO   27.62         10.52        69.54        32.66
2025-01-21  CDRE   38.76          6.63        76.29         4.33
2025-01-21   PDD  108.07          6.47       -27.22       113.33
2025-01-21  QBTS    5.40         -5.76        70.16       -53.43
2025-01-21   WBA   11.72        -10.05        48.86        81.20
2025-01-21   APA   23.99 

### TTM Squeeze

In [ ]:
# TTM SQUEEZE

res = collection.find(
    {"script":"daily_ttm_squeeze", "DATE":yesterday},
    {"_id":0, "script":0, "timestamp":0}
    )

df = pd.DataFrame(res)
df.set_index("DATE", inplace=True)
print(df)

            TICK  DAYS
DATE                  
2025-01-21   CAH    10
2025-01-21   MMM    46
2025-01-21  XPEV    27
2025-01-21   HWM     9
2025-01-21   VLO     7
...          ...   ...
2025-01-21  ISRG    32
2025-01-21   TPR     6
2025-01-21    ET    22
2025-01-21   DAL    10
2025-01-21   ETR    15

[72 rows x 2 columns]


In [ ]:

print('The END')

The END
